In [1]:
from ssd import build_ssd
from layers.box_utils import *
import os
import sys
import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
import argparse
import torchvision
import pickle
from layers import box_utils
from layers import Detect
from layers import functions
from layers import modules
import torch.nn.functional as F
from math import sqrt as sqrt
from itertools import product as product

from torch.autograd import Function
from layers.box_utils import decode, nms

In [2]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)
ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')

Loading weights into state dict...
Finished!


/home/shoghi/文件/電腦視覺深度學習馬拉松/Day29-31/Object Detection 程式導讀/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


## 默認Config檔案在data/config.py內

In [3]:
ssd_net.cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [4]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [5]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    ## 從min_ratio至max_ratio + 1每隔步驟= 17取一個值賦值給比。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [6]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                '''中心點'''
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                '''/self.image_size 就是在做normalization '''
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [7]:
PriorBox_Demo=PriorBox(cfg)

In [8]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


---

## Loss 如何設計

In [9]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [10]:
Use_cuda=True
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

----

## 基本設定

In [11]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=True
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
net=ssd_net

Loading weights into state dict...
Finished!


In [12]:
'''要不要使用gpu'''
# Use_cuda=True

'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    if Use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if not Use_cuda:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

'''使用GPU時可以開啟DataParallel，但當Input是不定大小時，要關掉'''
if Use_cuda:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
'''使用GPU時模型要轉成cuda'''
if Use_cuda:
    net = net.cuda()
    
batch_size_=1
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [13]:
'''輸入影像格式，假設batch size 為 4'''
image_in=torch.tensor(torch.rand(4,3,300,300),dtype=torch.float32)
'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

/home/shoghi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
epochs=300
iteration=1000

In [15]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in).type(torch.cuda.FloatTensor)
            y = [Variable(torch.tensor(label_0).type(torch.cuda.FloatTensor)),Variable(torch.tensor(label_1).type(torch.cuda.FloatTensor)),
                 Variable(torch.tensor(label_2).type(torch.cuda.FloatTensor)),Variable(torch.tensor(label_3).type(torch.cuda.FloatTensor))]        
            
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0)),Variable(torch.tensor(label_1)) 
                ,Variable(torch.tensor(label_2)),Variable(torch.tensor(label_3))]

        '''Forward Pass'''
        out = net(image_)
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out, y)
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
    torch.save(ssd_net.state_dict(),'weights/Ｗeights.pth')

/home/shoghi/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BBOX Regression Loss:  1.9469382533320672
Classification Loss:  8.117625653302229
BBOX Regression Loss:  1.7458090676201716
Classification Loss:  6.3071305310284655
BBOX Regression Loss:  1.6273702692102505
Classification Loss:  5.4615030924479155
BBOX Regression Loss:  1.4890334853419553
Classification Loss:  4.904060674596716
BBOX Regression Loss:  1.3554657589947734
Classification Loss:  4.468858862982856
BBOX Regression Loss:  1.2309845371010861
Classification Loss:  4.097168387895748
BBOX Regression Loss:  1.1201968985259847
Classification Loss:  3.762305621999912
BBOX Regression Loss:  1.0198244384041542
Classification Loss:  3.454992328749763
BBOX Regression Loss:  0.9272149468645638
Classification Loss:  3.1745275568079068
BBOX Regression Loss:  0.8457992747094898
Classification Loss:  2.920884028187505
BBOX Regression Loss:  0.7759090998357392
Classification Loss:  2.6921251682320033
BBOX Regression Loss:  0.715734475887852
Classification Loss:  2.4886843632768705
BBOX Regress

BBOX Regression Loss:  0.08744869629889798
Classification Loss:  0.30329701187434027
BBOX Regression Loss:  5.7744312680033214e-05
Classification Loss:  0.00020519627465142142
BBOX Regression Loss:  0.0023012541485918007
Classification Loss:  0.0002119859059651693
BBOX Regression Loss:  0.0021815542057824874
Classification Loss:  0.00022961916746916595
BBOX Regression Loss:  0.0017735805401941803
Classification Loss:  0.00024258759286668562
BBOX Regression Loss:  0.0014564688512904533
Classification Loss:  0.00024882793426513674
BBOX Regression Loss:  0.0012260355569948989
Classification Loss:  0.00025061633851793076
BBOX Regression Loss:  0.0010550256989075232
Classification Loss:  0.00024991186838301406
BBOX Regression Loss:  0.0009247375464299155
Classification Loss:  0.0002476280486142194
BBOX Regression Loss:  0.0008227231951958379
Classification Loss:  0.00024445763340702763
BBOX Regression Loss:  0.0007408857322594418
Classification Loss:  0.00024084011713663734
BBOX Regression 

BBOX Regression Loss:  0.000470028311319502
Classification Loss:  0.00016385217027771682
BBOX Regression Loss:  0.000498551915008567
Classification Loss:  0.00016304782926680224
BBOX Regression Loss:  0.0005142737250516616
Classification Loss:  0.00016258082303407156
BBOX Regression Loss:  0.0005177138402732577
Classification Loss:  0.00016233114238835684
BBOX Regression Loss:  0.0005150948725218084
Classification Loss:  0.0001620907014534797
BBOX Regression Loss:  0.0005107619015835671
Classification Loss:  0.00016179333301207728
BBOX Regression Loss:  0.0005058944199021864
Classification Loss:  0.00016143731068952105
BBOX Regression Loss:  0.0005009165558975676
Classification Loss:  0.0001610122902581543
BBOX Regression Loss:  0.0004959660336587856
Classification Loss:  0.00016053009033203125
BBOX Regression Loss:  2.973025146746767e-06
Classification Loss:  0.00010733251218442564
BBOX Regression Loss:  2.3496908599756554e-06
Classification Loss:  0.00010499777617277923
BBOX Regressi

BBOX Regression Loss:  0.00033235555414567834
Classification Loss:  8.937924627273801e-05
BBOX Regression Loss:  0.00032845114241183753
Classification Loss:  8.898715308029408e-05
BBOX Regression Loss:  0.00032463747704737417
Classification Loss:  8.858257485914602e-05
BBOX Regression Loss:  0.00032096698114672105
Classification Loss:  8.81690378136308e-05
BBOX Regression Loss:  0.00034330423905347055
Classification Loss:  8.776625398834948e-05
BBOX Regression Loss:  0.00036302445153766483
Classification Loss:  8.758493527441387e-05
BBOX Regression Loss:  0.0003651541763283529
Classification Loss:  8.75576631522473e-05
BBOX Regression Loss:  0.0003631517285996663
Classification Loss:  8.754967963933264e-05
BBOX Regression Loss:  0.0003599470099596663
Classification Loss:  8.75126336507751e-05
BBOX Regression Loss:  0.0003563327348995256
Classification Loss:  8.743782529599255e-05
BBOX Regression Loss:  0.0003526381982745328
Classification Loss:  8.732451520819098e-05
BBOX Regression Lo

...

BBOX Regression Loss:  1.694099431154199e-05
Classification Loss:  1.5758720897881053e-09
BBOX Regression Loss:  1.7412464776154516e-05
Classification Loss:  1.551995239942831e-09
BBOX Regression Loss:  1.7405255850351224e-05
Classification Loss:  1.6079086042228094e-09
BBOX Regression Loss:  1.7235060787059966e-05
Classification Loss:  1.5842628894548269e-09
BBOX Regression Loss:  1.70194701438924e-05
Classification Loss:  1.5613025577235975e-09
BBOX Regression Loss:  1.6788903464165137e-05
Classification Loss:  1.5389982354704034e-09
BBOX Regression Loss:  1.6557704758919742e-05
Classification Loss:  1.5173222039849047e-09
BBOX Regression Loss:  1.6331841744928557e-05
Classification Loss:  1.4962482844851144e-09
BBOX Regression Loss:  1.618858458240924e-05
Classification Loss:  1.4757517326428524e-09
BBOX Regression Loss:  1.6761159126925288e-05
Classification Loss:  1.4558091416611923e-09
BBOX Regression Loss:  1.688960893807515e-05
Classification Loss:  1.4363983531057098e-09
BBOX 

BBOX Regression Loss:  1.5446795991361763e-05
Classification Loss:  1.471719624083719e-09
BBOX Regression Loss:  1.5812762005967755e-05
Classification Loss:  1.5376932624047132e-09
BBOX Regression Loss:  1.5845621405927703e-05
Classification Loss:  1.5116306647368369e-09
BBOX Regression Loss:  1.577192263469296e-05
Classification Loss:  1.4864368203245563e-09
BBOX Regression Loss:  1.5645802760712525e-05
Classification Loss:  1.4620690035979242e-09
BBOX Regression Loss:  1.5543609151438167e-05
Classification Loss:  1.438487245475377e-09
BBOX Regression Loss:  1.5496644196186553e-05
Classification Loss:  1.4156541145948155e-09
BBOX Regression Loss:  1.5504073973197047e-05
Classification Loss:  1.3935345190542715e-09
BBOX Regression Loss:  1.5741933802125286e-05
Classification Loss:  1.3720955264534364e-09
BBOX Regression Loss:  1.577060829841367e-05
Classification Loss:  1.3513062002950512e-09
BBOX Regression Loss:  1.566735204465455e-05
Classification Loss:  1.331137451036916e-09
BBOX 

BBOX Regression Loss:  1.8166182006782796e-05
Classification Loss:  1.8021056621433294e-09
BBOX Regression Loss:  1.864347891400771e-05
Classification Loss:  1.7660635489004628e-09
BBOX Regression Loss:  1.8550072810861805e-05
Classification Loss:  1.7314348518631989e-09
BBOX Regression Loss:  1.8284517020940275e-05
Classification Loss:  1.6981380277889065e-09
BBOX Regression Loss:  1.7970960221197523e-05
Classification Loss:  1.666097687641946e-09
BBOX Regression Loss:  1.764983977897237e-05
Classification Loss:  1.7987684294356565e-09
BBOX Regression Loss:  1.7334195421677664e-05
Classification Loss:  1.7660635489004628e-09
BBOX Regression Loss:  1.703427738083707e-05
Classification Loss:  1.7345266998129548e-09
BBOX Regression Loss:  1.6979819406559486e-05
Classification Loss:  1.7040964068337802e-09
BBOX Regression Loss:  1.7563391630814625e-05
Classification Loss:  1.6747154343021634e-09
BBOX Regression Loss:  1.7513130751353358e-05
Classification Loss:  1.6463304269411098e-09
BBO

BBOX Regression Loss:  1.6175559625000664e-05
Classification Loss:  1.567381399649161e-09
BBOX Regression Loss:  1.5802492787420467e-05
Classification Loss:  1.5291525850235716e-09
BBOX Regression Loss:  1.544092913151456e-05
Classification Loss:  1.4927441901420577e-09
BBOX Regression Loss:  1.525380110727224e-05
Classification Loss:  1.4580292089759634e-09
BBOX Regression Loss:  1.685262326204165e-05
Classification Loss:  1.424892181499237e-09
BBOX Regression Loss:  1.7053751277930086e-05
Classification Loss:  1.393227910799254e-09
BBOX Regression Loss:  1.6908178659267307e-05
Classification Loss:  1.3629403475210094e-09
BBOX Regression Loss:  1.661897560113123e-05
Classification Loss:  1.3339416167226902e-09
BBOX Regression Loss:  1.6300104078971868e-05
Classification Loss:  1.3061511663743008e-09
BBOX Regression Loss:  1.597879881464507e-05
Classification Loss:  1.279495020121764e-09
BBOX Regression Loss:  1.569130173895785e-05
Classification Loss:  1.2539051197193288e-09
BBOX Regr

BBOX Regression Loss:  1.4366241888042424e-05
Classification Loss:  1.4242447975003732e-09
BBOX Regression Loss:  1.3985465685622062e-05
Classification Loss:  1.5177108623363355e-09
BBOX Regression Loss:  1.4032626317852013e-05
Classification Loss:  1.4717196240837193e-09
BBOX Regression Loss:  1.5158719684728163e-05
Classification Loss:  1.4284337527871393e-09
BBOX Regression Loss:  1.5127468732460994e-05
Classification Loss:  1.387621359850364e-09
BBOX Regression Loss:  1.4858804319254263e-05
Classification Loss:  1.3490763220767426e-09
BBOX Regression Loss:  1.4508596767744937e-05
Classification Loss:  1.3126147998584524e-09
BBOX Regression Loss:  1.4190671811209693e-05
Classification Loss:  1.2780723051253352e-09
BBOX Regression Loss:  1.51623995719801e-05
Classification Loss:  1.2453012203785316e-09
BBOX Regression Loss:  1.5804566519700868e-05
Classification Loss:  1.3245476616753473e-09
BBOX Regression Loss:  1.5680124349414174e-05
Classification Loss:  1.2922416211466802e-09
BB